In [14]:
import torch
import torch.nn as nn
import numpy as np

import matplotlib.pyplot as plt

#from cvx import CVX
from sketchcvx import sketchCVX

# Smooth L1-Regression

Regression with smooth L1-loss.
Given data points $a_1, \dots, a_n$ and observations $y_1, \dots, y_n$, the goal is to minimize the smooth L1-loss:
$$f(Ax) = \frac{1}{n} \sum_{i=1}^n \ell(a_i^\top x, y_i)\,,$$
where
$$\ell(w_i, y_i) = \begin{cases} 0.5 (w_i - y_i)^2\,, \quad &\text{if } |w_i-y_i| <  1\\ |w_i - y_i|-0.5 \quad &\text{otherwise}\end{cases}$$

We illustrate how to optimize in a low-dimensional subspace. First, we generate a toy example.

In [15]:
n, d, p = 80, 30, 1
A = torch.randn(n,d)
U, _, V = torch.svd(A)
Sigma = torch.Tensor([0.8**ii for ii in range(d)])
A = torch.mm(U, Sigma*V.T)
y = torch.randn(n, p)
lambda_ = 1.

In [16]:
def SmoothL1Loss(w, y):
    n = y.shape[0]
    crit = (torch.abs(w-y) < 1).double()
    return 1/n*(crit*(w-y)**2 + (1-crit)*(torch.abs(w-y)-0.5)).sum()

We solve the sketched program as follows.

In [23]:
#We choose: 
#       - the sketch size
#       - adaptive or oblivious sketch
#       - embedding : srht or gaussian
#       - the power in the power method
sketch_params = {'sketch_size': 5, 'adaptive': True, 'sketch': 'srht', 'power': 1}

# This creates an instance of the solver
cvx = sketchCVX(A, y, SmoothL1Loss, lambda_)

# In the next line, we sketch the data.
_, sketch_time = cvx.sketch(**sketch_params)

# We choose an algorithm and its parameter. Here, we use the subsampling Newton method.
# For the Newton method, we specify two batch sizes:
    # - one for computing a stochastic gradient
    # - one for computing a stochastic hessian
# 'params': 
    # lr = step size
    # tol = tolerance for solving H^{-1}*g using the conjugate gradient method
    # n_cg = max number of iterations of the CG method
optim_params = {'alg': 'newton', 
                'batch_sizes': [n, n], 
                'params': {'lr': .5, 
                           'tol': 1e-3, 
                           'n_cg': 20}}

# alternatively, use 'sgd', 'adagrad' or 'adam' (see the Pytorch documentation for more details)
#optim_params = {'alg': 'sgd', 
#                'batch_sizes': [n], 
#                'params': {'lr': .8, 'momentum': 0.9}}

# Then, run the following line to compute the approximate solution from the subspace method
xtilde, _, times_lowdim = cvx.iterative_method(n_iterations=10, **optim_params)

Compare the latter computational times with that of solving the high-dimensional program.

In [24]:
from cvx import CVX
cvx = CVX(A, y, SmoothL1Loss, lambda_)

optim_params = {'alg': 'newton', 
                'batch_sizes': [n, n], 
                'params': {'lr': .5, 
                           'tol': 1e-3, 
                           'n_cg': 20}}

x, _, times_highdim = cvx.solve(n_iterations=10, **optim_params)

dist = (torch.norm(x-xtilde) / torch.norm(x)).numpy()
print('distance between high-dim and low-dim solutions:', dist)
      

distance between high-dim and low-dim solutions: 0.04785702


In [25]:
print('time for highdimopt: ', np.cumsum(times_highdim)[-1])

print('time for lowdimopt: ', np.cumsum(times_lowdim)[-1])

time for highdimopt:  0.11011219024658203
time for lowdimopt:  0.03330731391906738
